In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
train_set = torchvision.datasets.FashionMNIST(root = r"C:\Users\Administrator\Desktop\机器学习，深度学习参考教材\deeplizard--pytorch神经网络入门\准备数据\data_FashionMNIST",
                                              train= True,download = True,transform = transforms.Compose([transforms.ToTensor()]))

train_loader = torch.utils.data.DataLoader(train_set,batch_size = 100)



torch.set_grad_enabled(True)

class Network(nn.Module):

    def __init__(self):
        super(Network,self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 1,out_channels=6,kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels = 12,kernel_size  =5)



        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=60)
        self.fc3 = nn.Linear(in_features=60,out_features=10)


    def forward(self,t):

        t = self.conv1(t)
        t = F.relu(t)
        t  = F.max_pool2d(t,kernel_size= 2,stride = 2)


        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size = 2,stride = 2)

        t = self.fc1(t.reshape(-1,12*4*4))
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.fc3(t)

        return t

network = Network()
updater = optim.SGD(network.parameters(),lr = 0.01)
# loss = F.cross_entropy(preds,labels)

def get_num_correct(preds,labels):
    return  (preds.argmax(dim = 1) == labels).sum()



for epoch  in range(10):
    train_loss = 0
    train_num_correct = 0
    for images,labels in train_loader:

        preds = network(images)
        train_num_correct += get_num_correct(preds,labels)
        loss = F.cross_entropy(preds,labels)
        train_loss += loss

        updater.zero_grad()
        loss.backward()


        updater.step()
    print("epoch=  ",{epoch},"train_loss = ",{train_loss},"correct_% = ",{train_num_correct/len(train_set)})



epoch=   {0} train_loss =  {tensor(1376.6747, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.1843)}
epoch=   {1} train_loss =  {tensor(970.8980, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.4587)}
epoch=   {2} train_loss =  {tensor(534.2161, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.6592)}
epoch=   {3} train_loss =  {tensor(469.8138, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.7006)}
epoch=   {4} train_loss =  {tensor(430.7572, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.7258)}
epoch=   {5} train_loss =  {tensor(401.1327, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.7460)}
epoch=   {6} train_loss =  {tensor(378.2709, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.7616)}
epoch=   {7} train_loss =  {tensor(359.5994, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.7749)}
epoch=   {8} train_loss =  {tensor(343.3848, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.7852)}
epoch=   {9} train_loss =  {tensor(329.3903, grad_fn=<AddBackward0>)} correct_% =  {tensor(0.7946)}

In [5]:
def get_all_preds(network,train_loader):
    all_preds = torch.tensor([])
    for batch in train_loader:
        images,labels = batch
        preds =network(images)

        all_preds = torch.cat((all_preds,preds),dim=0)
    return all_preds

with torch.no_grad():
    train_preds = get_all_preds(network,train_loader)


train_preds = train_preds.argmax(dim = 1)
print("train_preds——shape-----(6000,)?????",train_preds.shape)
# stacked = torch.cat((train_preds,train_set.targets),dim =1 )
stacked = torch.stack((train_preds,train_set.targets),dim =1 )

cmt = torch.zeros((10,10),dtype = torch.int64)
for p in stacked:
    j,k = p
    cmt[k,j] += 1

print(cmt)

train_preds——shape-----(6000,)????? torch.Size([60000])
tensor([[4879,   67,  252,  528,   36,   10,  139,    0,   88,    1],
        [  30, 5572,   48,  305,   34,    1,    6,    0,    3,    1],
        [  44,   12, 4666,   77,  850,   11,  213,    0,  127,    0],
        [ 182,  108,  152, 5293,  163,    6,   68,    0,   28,    0],
        [  10,   20, 1027,  340, 4344,    3,  190,    0,   65,    1],
        [   1,    0,    1,    4,    0, 5542,    0,  289,   33,  130],
        [1304,   25, 1998,  418,  759,    2, 1330,    0,  164,    0],
        [   0,    0,    0,    0,    0,  379,    0, 5341,    6,  274],
        [  24,    1,  120,   45,   34,   49,   18,   12, 5691,    6],
        [   0,    0,    0,    5,    1,  141,    0,  283,    6, 5564]])


In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(train_set.targets,train_preds.reshape(-1,1))
print(cm)

[[4879   67  252  528   36   10  139    0   88    1]
 [  30 5572   48  305   34    1    6    0    3    1]
 [  44   12 4666   77  850   11  213    0  127    0]
 [ 182  108  152 5293  163    6   68    0   28    0]
 [  10   20 1027  340 4344    3  190    0   65    1]
 [   1    0    1    4    0 5542    0  289   33  130]
 [1304   25 1998  418  759    2 1330    0  164    0]
 [   0    0    0    0    0  379    0 5341    6  274]
 [  24    1  120   45   34   49   18   12 5691    6]
 [   0    0    0    5    1  141    0  283    6 5564]]


In [11]:
import matplotlib.pyplot as plt
from sklearn import  metrics
names = ("1","2","3","4","5","6","7","8","9","10")
plt.figure(figsize=(5,5))
metrics.plot_confusion_matrix(cm,)

C:\Users\Administrator\miniconda3\envs\my_env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


ValueError: plot_confusion_matrix only supports classifiers

<Figure size 500x500 with 0 Axes>